# 🔮 cellspell — SPARQL Spell (Colab)

Run SPARQL queries against RDF files (via rdflib) or SPARQL endpoints (Wikidata, Fuseki, etc.).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sreent/jupyter-query-magics/blob/main/examples/colab_sparql.ipynb)

## Setup

In [ ]:
!pip install "cellspell[sparql]" -q

In [ ]:
%load_ext cellspell.sparql

## Part 1: RDF Files (via rdflib)

Query a Turtle file with `--file`.

In [ ]:
%%writefile scientists.ttl
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:einstein a foaf:Person ;
    foaf:name "Albert Einstein" ;
    ex:field "Physics" ;
    ex:birthYear "1879"^^xsd:integer ;
    ex:nationality "German" .

ex:curie a foaf:Person ;
    foaf:name "Marie Curie" ;
    ex:field "Chemistry" ;
    ex:birthYear "1867"^^xsd:integer ;
    ex:nationality "Polish" .

ex:turing a foaf:Person ;
    foaf:name "Alan Turing" ;
    ex:field "Computer Science" ;
    ex:birthYear "1912"^^xsd:integer ;
    ex:nationality "British" .

ex:hopper a foaf:Person ;
    foaf:name "Grace Hopper" ;
    ex:field "Computer Science" ;
    ex:birthYear "1906"^^xsd:integer ;
    ex:nationality "American" .

ex:darwin a foaf:Person ;
    foaf:name "Charles Darwin" ;
    ex:field "Biology" ;
    ex:birthYear "1809"^^xsd:integer ;
    ex:nationality "British" .

In [ ]:
# List all scientists
%%sparql --file scientists.ttl
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX ex: <http://example.org/>
SELECT ?name ?field WHERE {
    ?person a foaf:Person ;
            foaf:name ?name ;
            ex:field ?field .
}
ORDER BY ?name

In [ ]:
# Computer scientists only
%%sparql --file scientists.ttl
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX ex: <http://example.org/>
SELECT ?name ?year WHERE {
    ?person a foaf:Person ;
            foaf:name ?name ;
            ex:field "Computer Science" ;
            ex:birthYear ?year .
}

In [ ]:
# Born before 1900
%%sparql --file scientists.ttl
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX ex: <http://example.org/>
SELECT ?name ?year WHERE {
    ?person foaf:name ?name ;
            ex:birthYear ?year .
    FILTER(?year < 1900)
}
ORDER BY ?year

In [ ]:
# Count by field
%%sparql --file scientists.ttl
PREFIX ex: <http://example.org/>
SELECT ?field (COUNT(?person) AS ?count) WHERE {
    ?person ex:field ?field .
}
GROUP BY ?field
ORDER BY DESC(?count)

In [ ]:
# ASK query — any American scientists?
%%sparql --file scientists.ttl
PREFIX ex: <http://example.org/>
ASK {
    ?person ex:nationality "American" .
}

### Graph Info & Reset

In [ ]:
# Show loaded graph info
%sparql

In [ ]:
# Clear loaded graph
%sparql --reset

### Query Multiple Files with `--files`

In [ ]:
%%writefile planets.ttl
@prefix ex: <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:earth rdfs:label "Earth" ; ex:radius 6371 .
ex:mars  rdfs:label "Mars"  ; ex:radius 3390 .
ex:jupiter rdfs:label "Jupiter" ; ex:radius 69911 .

In [ ]:
# Query across both scientists and planets
%%sparql --files scientists.ttl,planets.ttl
PREFIX ex: <http://example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
SELECT ?label WHERE {
    { ?s foaf:name ?label }
    UNION
    { ?s rdfs:label ?label }
}
ORDER BY ?label

## Part 2: SPARQL Endpoints (Wikidata)

Query any SPARQL endpoint with `--endpoint`. No extra dependencies needed.

In [ ]:
# Query Wikidata — list planets in the solar system
%%sparql --endpoint https://query.wikidata.org/sparql
SELECT ?planet ?planetLabel WHERE {
    ?planet wdt:P31 wd:Q634 .
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}

In [ ]:
# Query Wikidata — top 10 countries by population
%%sparql --endpoint https://query.wikidata.org/sparql
SELECT ?country ?countryLabel ?population WHERE {
    ?country wdt:P31 wd:Q6256 ;
             wdt:P1082 ?population .
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
}
ORDER BY DESC(?population)
LIMIT 10

## Access Results in Python

The last query result is stored in `_sparql` as a list of dicts.

In [ ]:
%%sparql --file scientists.ttl
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX ex: <http://example.org/>
SELECT ?name ?field WHERE {
    ?person foaf:name ?name ;
            ex:field ?field .
}
ORDER BY ?name

In [ ]:
# Access the result as a list of dicts
for row in _sparql:
    print(f"{row['name']} — {row['field']}")